## **Maestría en Inteligencia Artificial Aplicada**
### **Curso: Inteligencia Artificial y Aprendizaje Automático**
#### Tecnológico de Monterrey
####P rof Luis Eduardo Falcón Morales

### **Adtividad de la Semana 8**
#### **Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Esta Actividad deberás resolverla de manera individual.**

**Nombre y matrícula:**

*   Marcos Eduardo García Ortiz
*   A01276213


In [2]:
# Incluye aquí todas las librerías y paquetes que requieras.

from google.colab import drive
drive.mount('/content/drive')

import os
DIR = "/content/drive/MyDrive/Maestria/Segundo Trimestre/IAyAA/Actividad 6/"
os.chdir(DIR)

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.decomposition import TruncatedSVD

Mounted at /content/drive


### **Liga de Datos de la UCI "Restaurant & consumer data".**
### **Descarga "RCdata.zip" y de ahí necesitaremos los archivos "rating_final.csv" y "geoplaces2.csv".**

https://archive.ics.uci.edu/dataset/232/restaurant+consumer+data



In [ ]:
# Descarga los archivos de la página de la UCI a partir de los cuales generaremos
# nuestras matrices de utilidad.

data1 = pd.read_csv("rating_final.csv", header='infer', sep=",")
data2 = pd.read_csv("geoplaces2.csv", header='infer',  encoding='latin-1')

print(data1.shape, data2.shape)


(1161, 5) (130, 21)


## **Ejercicio - 1**

### **Explica cuál es el propósito de usar el argumento "latin-1" al cargar el segundo archivo.**

++++++++ Inicia la sección de agregar texto: +++++++++++

El parametro de encoding dentro de la función `read_csv` permite definir la codificación UTF (Unicode Transformation) que se usara para el archivo en este caso latin-1 la cual es la forma de llamar al estandar ISO 8859-1

Esta norma define la codificación del alfabeto español, incluyendo los diacríticos (como letras acentuadas, ñ, ç), y letras especiales (como ß, Ø), necesarios para la escritura de las siguientes lenguas originarias de Europa occidental: afrikáans, alemán, español, catalán, euskera, danés, gaélico escocés, feroés, francés, gaélico irlandés, gallego, inglés, islandés, italiano, neerlandés, noruego, portugués y sueco.

Sin especificar esta codificación el compilador sera incapaz de leer el archivo o en caso de hacerlo asignara simbolos diferentes a esos caracteres especiales

++++++++ Termina la sección de agregar texto. +++++++++++

In [ ]:
# Del primer archivo obtenemos una matriz con 3 evaluaciones de los restaurantes:
# general, comida y servicio.
# Las evaluaciones pueden ser 0, 1 o 2, siendo 0 la menor calificación y 2 la
# mayor calificación:

data1.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [ ]:
# Del segundo archivo obtenemos información diversa de cada restaurante:

data2.head(2)

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none


In [ ]:
# De data1 no requeriremos "rating" y de data2 solo necesitamos "placeID" y "name":
# Definimos la lista y matrices con los factores que necesitamos:

lista_data1 = ['userID','placeID','food_rating','service_rating']
lista_data2 = ['placeID','name']

data1a = data1[lista_data1]
data2a = data2[lista_data2]

## **Ejercicio - 2**

In [ ]:
# Define el DataFrame que conjunte la información de las dos DataFrame data1a
# y data2a en uno solo, a través de la columna común "placeID" y que
# llamaremos "df_combinado":



# ************* Inlcuye aquí tu código:*****************************

df_combinado = pd.merge(data1a, data2a, on="placeID", how="inner")

# *********** Aquí termina la sección de agregar código *************


# Despleguemos la dimensión y los primeros renglones de este DataFrame:

print(df_combinado.shape)
df_combinado.head()

(1161, 5)


,userID,placeID,food_rating,service_rating,name
0,U1077,135085,2,2,Tortas Locas Hipocampo
1,U1108,135085,2,1,Tortas Locas Hipocampo
2,U1081,135085,2,1,Tortas Locas Hipocampo
3,U1056,135085,2,2,Tortas Locas Hipocampo
4,U1134,135085,1,2,Tortas Locas Hipocampo


## **Ejercicio - 3**

In [ ]:
#    Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación de la comida (food_rating). La llamaremos "UtMx_food".


# ************* Inlcuye aquí tu código:*****************************


UtMx_food = df_combinado.pivot_table(values='food_rating', index='name', columns='userID', fill_value=0)


# *********** Aquí termina la sección de agregar código *************


print('Dimensión de la matriz de Utilidad sobre la evaluación de la comida:')
print('(restaurantes, usuarios) =', (UtMx_food.shape))
UtMx_food

Dimensión de la matriz de Utilidad sobre la evaluación de la comida:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,2,0,...,0,0,0,0,0,1,0,0,0,0
Cafe Chaires,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tacos de barbacoa enfrente del Tec,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tacos de la estacion,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tacos los volcanes,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## **Ejercicio - 4**

### **En la Factorización SVD la cantidad de valores singulares será menor o igual al menor valor de los renglones o columnas de la matriz de utilidad UtMx.**

### **La factorización SVD de una matriz $A$ tiene la forma:**

$A_{m\times n} = U_{m\times m}\Sigma_{m\times n}V_{n\times n}^T$

In [ ]:
# Apliquemos la transformación SVD para obtener la matriz de las variables
# latentes de los restaurantes en relación a la evalución de los usuarios con
# respecto a la comida. Deberás obtener la factorización SVD con el máximo de
# comonentes y por lo tanto el máximo de valores singulares posible.
# Asigna a la variable "nc_food" el valor del máximo número de valores singulares
# que podemos obtener de la utilidad que generamos en el ejercicio anterior.


# ************* Inlcuye aquí tu código:*****************************


nc_food = 137     # número de componentes


# *********** Aquí termina la sección de agregar código *************


# Inicializamos y aplicamos la factorización SVD en relación a la
# evaluación de la comida, para determinar la cantidad de componentes
# que podemos utilizar con un 90% de explicación de la variabilidad
# de dichas componentes:

SVD_food = TruncatedSVD( n_components = nc_food )
SVD_food.fit(UtMx_food)

# Determinemos la cantidad de componentes que explican un 90%
# de la variabilidad acumulada de cada componente:

for j in range(nc_food):
  if SVD_food.explained_variance_ratio_[0:j].sum() > 0.90:
    break

# Usaremos esta cantidad de componentes para las recomendaciones basadas
# en la calificación de la comida:
N_food = j-1

print('Total de valores singulares basados en evaluación de la :', nc_food)
print('Valor de truncamiento al 90% de dicha variabilidad:',N_food)


Total de valores singulares basados en evaluación de la : 137
Valor de truncamiento al 90% de dicha variabilidad: 51


In [ ]:
# Usando la métrica de correlación de Pearson, obtendremos las 10 mejores
# recomendaciones de este modelo no supervisado con base a la información de
# alguien que desea obtener las similidades com el restaurante "Restaurante Pueblo
# Bonito" y con la cota de truncamiento obtenido en el ejercicio anterior.
# Para ello obtendremos las correlaciones positivas y de ahí deberás seleccionar
# las diez mejores.

# Factorización SVD:
SVD_food = TruncatedSVD(n_components = N_food)
resultant_matrix_food = SVD_food.fit_transform(UtMx_food)

# Matriz de correlación de Pearson:
corr_mat_food = np.corrcoef(resultant_matrix_food)

# Restaurante de referencia:
restaurante_de_referencia = "Restaurante Pueblo Bonito"
nombres_rest = UtMx_food.T.columns  # nombres de restaurantes
idx_rest = list(nombres_rest).index(restaurante_de_referencia)
corr_rest = corr_mat_food[idx_rest] # Vector de Correlación del RinconHuasteco

# Buscando las correlaciones positivas:
idx = (corr_rest>0)
mejores_sim_food = list()
for i in range(len(nombres_rest[idx])):
  mejores_sim_food.append((corr_rest[idx][i], nombres_rest[idx][i]))

print('Total de similaridades positivas encontradas:', len(mejores_sim_food))

print('Algunos de los resultados encontrados:')
mejores_sim_food[0:7]

Total de similaridades positivas encontradas: 71
Algunos de los resultados encontrados:


[(0.007119067595729353, 'Arrachela Grill'),
 (0.02067917095453551, 'Cabana Huasteca'),
 (0.03010569846739319, 'Cafe Chaires'),
 (0.01885800082925752, 'Cafeteria cenidet'),
 (0.2238923257006633, 'Cafeteria y Restaurant El Pacifico'),
 (0.013454502359705146, 'Cenaduria El RincÃ³n de Tlaquepaque'),
 (0.005842844857144624, 'Chilis Cuernavaca')]

## **Ejercicio - 5**

In [ ]:
# Ordenar la lista de recomendaciones "mejores_sim_food" encontrada en el paso
# anterior de manera descendente y llamaremos "mejores_sim_food_ordenadas".
#
# NOTA: Puedes incluir a continuación las líneas de código que requieras.

# ************* Inlcuye aquí tu código:*****************************

mejores_sim_food_ordenadas = sorted(mejores_sim_food, reverse=True)

# *********** Aquí termina la sección de agregar código *************


# Desplegamos las 10 mejores similitudes encontradas de manera descendente:
print('Similitudes con base a la evaluación de la comida con mayores valores de correlación:')
for k in range(1,11):
  print('%d> %s' % (k, mejores_sim_food_ordenadas[k]))

Similitudes con base a la evaluación de la comida con mayores valores de correlación:
1> (0.7959018204580454, 'Restaurante la Estrella de Dima')
2> (0.6947498983528219, 'pizza clasica')
3> (0.6674416525606569, 'Restaurante Guerra')
4> (0.543336956695148, 'El Club')
5> (0.3666494942908652, 'Pizzeria Julios')
6> (0.31133693944698027, 'Restaurant Oriental Express')
7> (0.30966475444977076, 'Restaurante El Cielo Potosino')
8> (0.3029350952870184, 'Hamburguesas saul')
9> (0.2849447597086528, 'Rincon Huasteco')
10> (0.2683306971311213, 'la Cochinita Pibil Restaurante Yucateco')


## **Ejercicio - 6**

### **De manera análoga y usando ahora la evaluación con respecto al servicio (service_rating), encuentrarás ahora los diez restaurantes con mayor similaridad al "Restaurante Pueblo Bonito" y con respecto a la métrica de similaridad de Pearson.**

### **Deberás usar la cantidad de componentes necesarias para que la varabilidad de la factorización SVD quede explicada en un 90%.**

### **a.  Para ello define primero la matriz de utilidad, UtMx_service, donde los renglones son los nombres de los restaurantes, las columnas los usuarios y las entradas las evaluaciones con respecto al servicio recibido en el restaurante.**





In [ ]:
# a. Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación del servicio (service_rating). La llamaremos "UtMx_service".

# ************* Inlcuye aquí tu código:*****************************

UtMx_service = df_combinado.pivot_table(values='service_rating', index='name', columns='userID', fill_value=0)

# *********** Aquí termina la sección de agregar código *************

print('Dimensión de la matriz de Utilidad sobre la evaluación del servicio:')
print('(restaurantes, usuarios) =', (UtMx_service.shape))
UtMx_service.head()

Dimensión de la matriz de Utilidad sobre la evaluación del servicio:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
Cafe Chaires,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# b. Incluye el código necesario para obtener y desplegar los 10 restaurantes con
#    mayor similaridad de Pearson al "Restaurante Pueblo Bonito", y usando solo
#    el 90% de las componentes que explican la varianza de la matriz UtMx_service.
#    Desplegarlos en orden descendente, sin incliur el restaurante de referencia.

# ************* Inlcuye aquí tu código:*****************************

nc_service = 137     # número de componentes

# *********** Aquí termina la sección de agregar código *************


SVD_service = TruncatedSVD( n_components = nc_service )
SVD_service.fit(UtMx_service)

# Determinemos la cantidad de componentes que explican un 90%
# de la variabilidad acumulada de cada componente:

for j in range(nc_food):
  if SVD_service.explained_variance_ratio_[0:j].sum() > 0.90:
    break

# Usaremos esta cantidad de componentes para las recomendaciones basadas
# en la calificación de la comida:
N_service = j-1

print('Total de valores singulares basados en evaluación de la :', nc_service)
print('Valor de truncamiento al 90% de dicha variabilidad:',N_service)

# *********** Aquí termina la sección de agregar código *************

Total de valores singulares basados en evaluación de la : 137
Valor de truncamiento al 90% de dicha variabilidad: 51


In [ ]:
# Factorización SVD:
SVD_serivce = TruncatedSVD(n_components = N_service)
resultant_matrix_service = SVD_service.fit_transform(UtMx_service)

# Matriz de correlación de Pearson:
corr_mat_service = np.corrcoef(resultant_matrix_service)

# Restaurante de referencia:
restaurante_de_referencia = "Restaurante Pueblo Bonito"
nombres_rest = UtMx_service.T.columns  # nombres de restaurantes
idx_rest = list(nombres_rest).index(restaurante_de_referencia)
corr_rest = corr_mat_service[idx_rest] # Vector de Correlación

# Buscando las correlaciones positivas:
idx = (corr_rest>0)
mejores_sim_service = list()
for i in range(len(nombres_rest[idx])):
  mejores_sim_service.append((corr_rest[idx][i], nombres_rest[idx][i]))

print('Total de similaridades positivas encontradas:', len(mejores_sim_service))

print('Algunos de los resultados encontrados:')
mejores_sim_service[0:7]

Total de similaridades positivas encontradas: 109
Algunos de los resultados encontrados:


[(0.1846168007179288, 'Abondance Restaurante Bar'),
 (0.03834043540713542, 'Cabana Huasteca'),
 (0.0005065432989636979, 'Cafe Chaires'),
 (0.0007576258792744015, 'Cafeteria cenidet'),
 (0.17687849569001937, 'Cafeteria y Restaurant El Pacifico'),
 (0.001921451329392907, 'Carnitas Mata  Calle 16 de Septiembre'),
 (3.8401876961664725e-05, 'Carreton de Flautas y Migadas')]

In [ ]:
# ************* Inlcuye aquí tu código:*****************************

mejores_sim_service_ordenadas = sorted(mejores_sim_service, reverse=True)

# *********** Aquí termina la sección de agregar código *************

# Desplegamos las 10 mejores similitudes encontradas de manera descendente:
print('Similitudes con base a la evaluación de la comida con mayores valores de correlación:')
for k in range(1,11):
  print('%d> %s' % (k, mejores_sim_service_ordenadas[k]))

Similitudes con base a la evaluación de la comida con mayores valores de correlación:
1> (0.5773584679096652, 'pizza clasica')
2> (0.5239058304126724, 'El Club')
3> (0.436393420345008, 'Restaurante la Estrella de Dima')
4> (0.40026112492040705, 'Restaurante Guerra')
5> (0.3573629547365425, 'Restaurant Orizatlan')
6> (0.34804004420320384, 'Restaurante El Cielo Potosino')
7> (0.33337001114058, 'la Cochinita Pibil Restaurante Yucateco')
8> (0.29455671874921474, 'puesto de tacos')
9> (0.2917325371545275, 'Hamburguesas saul')
10> (0.28921563564922426, 'Restaurant Oriental Express')


## **Ejercicio - 7**

### **Incluye tus comentarios y conclusiones de la actividad. En particular indica cuáles restaurantes encontraste con correlaciones altas tanto para la calificación de la comida, como del servicio.**

++++++++ Inicia la sección de agregar texto: +++++++++++

En esta actividad se realizó un sistema de recomendación para restaurantes, comenzamos analizando el data set en el cual encontramos un total de 138 usuarios y 129 restaurantes, donde los usuarios emitieron diferentes puntuaciones y se contaba con diversos datos, se decidió centrar el sistema en base a las metricas de `food_rating` y `service_rating`.

Los datos se encontraban en dos archivos diferentes por lo que se realizo el merge de estos usando como llave `userID` para posteriormente realizar la matriz de utilidad donde definimos los restaurantes como filas y los usuarios como columnas.

Posteriormente se aplico la reducción de dimensionalidad mediante SVD para lograr simplificar el procesamiento, buscando el número de componentes que alcancen un 90% de variabilidad de los datos.

Se aplico el análisis con las dos métricas para saber que restaurantes recomendar a una persona que le gusto el Restaurante Pueblo Bonito en base a `food_rating` se recomendaron los siguientes:

1. Restaurante la Estrella de Dima
2. pizza clasica
3. Restaurante Guerra
4. El Club
5. Pizzeria Julios
6. Restaurant Oriental Express
7. Restaurante El Cielo Potosino
8. Hamburguesas saul
9. Rincon Huasteco
10. la Cochinita Pibil Restaurante Yucateco

Mediante una matriz de correlación de pearson y la reducción SVD se obtuvieron los restaurantes con mayor correlación ahora se realizo el mismo análisis mediante la variable `service_rating` obteniendo los siguientes restaurantes:


1. pizza clasica
2. El Club
3. Restaurante la Estrella de Dima
4. Restaurante Guerra
5. Restaurant Orizatlan
6. Restaurante El Cielo Potosino
7. la Cochinita Pibil Restaurante Yucateco
8. puesto de tacos
9. Hamburguesas saul
10. Restaurant Oriental Express

Los sistemas de recomendación pueden volverse más especificos donde una posible mejora es lograr organizar los restaurante en base a su tipo: Mariscos, Hamburguesas, Tacos para así recomendar en base a estos gustos

++++++++ Termina la sección de agregar texto. +++++++++++

>> **Fin de la Actividad de Sistemas de Recomendación**